In [14]:
from bs4 import BeautifulSoup
import requests
import xml.etree.ElementTree as ET
import pymongo
import pandas as pd
from sqlalchemy import create_engine

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_air_pollution'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
tables = soup.find_all('table', class_='wikitable sortable')
air_pollution_table = tables[0]
root = ET.Element("table_data")

In [4]:
headings = air_pollution_table.find_all('th')
headings_element = ET.SubElement(root, "headings")
for heading in headings:
    heading_text = heading.get_text(strip=True)
    heading_element = ET.SubElement(headings_element, "heading")
    heading_element.text = heading_text

In [5]:
rows = air_pollution_table.find_all('tr')[1:]  # Skip headings
rows_element = ET.SubElement(root, "rows")

for row in rows:
    row_element = ET.SubElement(rows_element, "row")
    cells = row.find_all(['td', 'th'])
    for cell in cells:
        cell_text = cell.get_text(strip=True)
        cell_element = ET.SubElement(row_element, "cell")
        cell_element.text = cell_text

In [7]:
tree = ET.ElementTree(root)

with open('air_pollution.xml', "wb") as xml_f:
    tree.write(xml_f, xml_declaration=True)

In [8]:
client = pymongo.MongoClient('192.168.56.30', 27017)
db = client.climate
collection = db.air_pollution_collection

with open('air_pollution.xml', 'rb') as xml_f:
    xml_data = xml_f.read()
    document = {'air_pollution': xml_data}
    collection.insert_one(document)

In [9]:
document = collection.find_one()
xml_data = document['air_pollution']
root = ET.fromstring(xml_data)
headings = [heading.text for heading in root.findall('.//headings/heading')]

In [16]:
data = []

for row in root.findall('.//rows/row'):
    row_data = [cell.text for cell in row.findall('.//cell')]
    data.append(row_data)

air_poll_df = pd.DataFrame(data, columns=headings)
air_poll_df

,Rank,Country/Region,2022,2021,2020,2019,2018,Population
0,1,Chad,89.7,75.9,--,--,--,"17,179,740"
1,2,Iraq,80.1,49.7,--,39.6,--,"43,533,592"
2,3,Pakistan,70.9,66.8,59,65.8,74.3,"231,402,117"
3,4,Bahrain,66.6,49.8,39.7,46.8,59.8,"1,463,265"
4,5,Bangladesh,65.8,76.9,77.1,83.3,97.1,"169,356,251"
...,...,...,...,...,...,...,...,...
126,127,"Bonaire, Saint Eustatius and Saba",3.3,5.1,--,--,--,"26,221"
127,128,Bermuda,3,--,--,--,--,"63,867"
128,129,U.S. Virgin Islands,2.9,4.5,3.7,3.5,--,"105,870"
129,130,French Polynesia,2.5,--,--,--,--,"304,032"


In [17]:
#Upload dataframe to postgres


DataFrame uploaded to PostgreSQL successfully.


In [ ]:
#(null values)
# maybe merge on air_pollution or global temps country/region for output table